# Librerías utilizadas

In [7]:
%pylab
#pylab.rcParams['figure.figsize'] = (10, 6)
style.use('default')

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Funciones

In [39]:
def GaussianNoise(SNR,Ps,M,N):
    """
    Recibe una potencia de señal Ps y una relación señal-ruido deseada
    SNR y retorna un vector Nu correspondiente a muestras del ruido 
    blanco gaussiano a sumarle a la señal
    """
    mean_W = 0
    var_W = Ps/(10**(SNR/10))
    std_W = np.sqrt(var_W)
    W = np.asmatrix(np.random.normal(mean_W, std_W, size=(M,N)))
    return (W)

def print_matrix(list_of_list):
    number_width = len(str(max([max(i) for i in list_of_list])))
    cols = max(map(len, list_of_list))
    output = '+'+('-'*(number_width+2)+'+')*cols + '\n'
    for row in list_of_list:
        for column in row:
            output += '|' + ' {:^{width}d} '.format(column, width = number_width)
        output+='|\n+'+('-'*(number_width+2)+'+')*cols + '\n'
    return output

def plotanim(Data,X,step=0.02,ylim=[]):
    plt.ion()
    fig, ax = plt.subplots(1,1)
    plt.ylim((10 ** -3, Data.max()))
    plt.yscale('log')
    line, = ax.plot(X,Data[:,0])
    fig.canvas.draw()
    for i in range(Data.shape[1]):
        plt.pause(step)
        line.set_ydata(Data[:,i])
        fig.canvas.draw()
    plt.ioff()

def doamusic_samples(Fc,s_amp,s_freq,t,t0,d,theta,M,D,SNR):
    # Definición de señales y muestras
    # Genero la matriz F de tamaño (D,N)
    lambda_c = c / Fc
    K = 2 * np.pi / (lambda_c)
    F = np.asmatrix(np.empty((D, N)))
    for i in range(0, D):
        F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

    # Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$
    A = np.asmatrix(np.empty((M, D), dtype='complex'))
    for i in range(0, M):
        for j in range(0, D):
            A[i, j] = np.e ** (-1j * i * K[j] * d * np.cos(theta[j]))

    # Genero el vector W de tamaño (M,1,N) de ruido $W\sim N(0,1)$
    Ps = 0
    for i in range(0,D):
        Ps = Ps + (s_amp[i]**2)/2
    W = GaussianNoise(SNR,Ps,M,N)

    # Genero el vector de muestras $X=A \times F + W$ de tamaño (M,N)
    X = np.asmatrix(np.empty((M,N),dtype=complex))
    X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
    for n in range (0,N):
        F_matrix = F[:,n]
        W_matrix = W[:,n]
        X_matrix = A @ F_matrix + W_matrix
        X[:, n] = X_matrix

    return X

def doamusic_estimation(X, K_est, theta_est):
    M = X.shape[0]
    N = X.shape[1]

    #%% Calculo el vector de covarianza S
    S = np.asmatrix(np.empty((M,M),dtype=complex))
    for n in range (0,N):
        X_matrix=np.asmatrix(X[:,n])
        S = S + (1/N)*(X_matrix @ X_matrix.H)

    # Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$
    [aval, avec] = eig(S)
    S0 = np.identity(M)
    p = aval.argsort()
    aval=np.abs(aval[p])
    avec=avec[:,p]
    aval_min = aval[0]

    # Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$
    Q = 0
    umbral = 2 * aval_min
    for i in range (0,M):
        if aval[i]-aval_min<umbral:
            Q += 1
    D_est = M - Q

    # Formo la matriz de subespacio de ruido $E_N$
    EN = np.asmatrix(avec[:,0:Q])

    #%% Evalúo la función $P_{MU}$ para distintas frecuencias de portadora y distintos ángulos de arribo
    a = np.asmatrix(np.empty((M, 1), dtype=complex))
    P_MU = np.empty((theta_est.size), dtype=complex)
    for i in range(0, theta_est.size):
        for k in range(0,M):
            a[k] = np.e ** (-1j * k * K_est * d * np.cos(theta_est[i]))
            P_MU[i] = 1 / (a.H @ EN @ EN.H @ a)      
    return P_MU

# Definición de constantes y unidades

In [9]:
kHz = 10 ** 3
MHz = 10 ** 6
GHz = 10 ** 9
ms = 10 ** -3
c = 3 * 10 ** 8
km = 10 ** 3

# Definición de variables

In [45]:
M = 8  # Número de sensores
D = 1  # Número de señales
N = 1000 # Número de snapshots

fs = 64 * MHz

T = 5 * ms # Tiempo de toma de muestras
t0 = 0

dist = 100  # Distancia en metros entre el emisor y la antena receptora
Tx_time = np.linspace(0, 15, num=1000)  # Tiempo del tracking
Tx_vel = 50 * km /3600 # Velocidad del transmisor en m/s
Tx_x = -100 + Tx_vel*Tx_time # Posición x del transmisor
Tx_y = 100  # Posición y del transmisor

theta_deg = 90 + np.arctan(Tx_x / Tx_y)*180/np.pi  #DOA en grados
theta = theta_deg * np.pi / 180         #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([50])
s_freq = np.array([440,1000,30000])

SNR = 7
t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)


In [113]:
theta_est = np.empty(N)
Fc_est = (1 + (((rand(N)-0.5)/0.5) * 0.1)) * Fc

# Primer estimación de ángulo
K_est = 2 * np.pi * Fc_est/c
theta_test = np.linspace(0, np.pi ,num=100)
X = doamusic_samples(Fc, s_amp, s_freq, t,0, d, array([theta[0]]), M, D, SNR)
P_MU = doamusic_estimation(X, K_est[0], theta_test)
theta_est[0] = theta_test[where(P_MU==max(P_MU))]

for i in range (1,N):
    theta_test = np.linspace(0.9,1.1,num=20) * theta_est[i-1]
    X = doamusic_samples(Fc, s_amp, s_freq, t,0, d, array([theta[i]]), M, D, SNR)
    P_MU = doamusic_estimation(X, K_est[i], theta_test)
    theta_est[i] = theta_test[where(P_MU==max(P_MU))]


In [116]:
figure()
plot(Tx_time,theta_est * 180 / np.pi)
plot(Tx_time,theta_deg)
xlabel('Tiempo de simulación [s]')
ylabel('AOA [°]')
show()